#Data preparation

In [8]:
!pip install transformers

In [9]:
import tensorflow as tf
#TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("TPU not activated")
    strategy = tf.distribute.MirroredStrategy() # Works on CPU, single GPU and multiple GPUs in a single VM.

TPU not activated


In [10]:
import pandas as pd
import numpy as np
import os

#Obtaining the datasets
train = pd.read_csv('train.csv')
final_test=pd.read_csv('test.csv')
# Some quick visualizations
print(train.head())
print(final_test.head())

           id                                            premise  \
0  5130fd2cb5  and these comments were considered in formulat...   
1  5b72532a0b  These are issues that we wrestle with in pract...   
2  3931fbe82a  Des petites choses comme celles-là font une di...   
3  5622f0c60b  you know they can't really defend themselves l...   
4  86aaa48b45  ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...   

                                          hypothesis lang_abv language  label  
0  The rules developed in the interim were put to...       en  English      0  
1  Practice groups are not permitted to work on t...       en  English      2  
2              J'essayais d'accomplir quelque chose.       fr   French      0  
3  They can't defend themselves because of their ...       en  English      0  
4    เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร       th     Thai      1  
           id                                            premise  \
0  c6d58c3f69  بکس، کیسی، راہیل، یسعیاہ، کی

#Pre Processing

In [11]:
import nltk
from nltk.corpus import gutenberg, stopwords, wordnet
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
import re
from bs4 import BeautifulSoup
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
def lower_case(text):
  return text.lower()
def rem_lines(text):
    return text.strip().replace('\n', ' ')
def rem_tags(text):
  return BeautifulSoup(text, "html.parser").get_text()

In [13]:
# Take out the id and target columns
features_train = train.drop(columns=['language','id','label', 'lang_abv'] )
features_test = final_test.drop(columns=['language','id', 'lang_abv'] )
# Concatenate all
train['features'] = features_train.astype(str).apply(' '.join, axis=1)
final_test['features']=features_test.astype(str).apply(' '.join, axis=1)
print(train['features'])
print(final_test['features'])

0        and these comments were considered in formulat...
1        These are issues that we wrestle with in pract...
2        Des petites choses comme celles-là font une di...
3        you know they can't really defend themselves l...
4        ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...
                               ...                        
12115    The results of even the most well designed epi...
12116    But there are two kinds of  the pleasure of do...
12117    The important thing is to realize that it's wa...
12118    At the west end is a detailed model of the who...
12119    For himself he chose Atat??rk, or Father of th...
Name: features, Length: 12120, dtype: object
0       بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...
1       هذا هو ما تم نصحنا به. عندما يتم إخبارهم بما ي...
2       et cela est en grande partie dû au fait que le...
3       与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp IMA与其他组织合作，因为...
4       Она все еще была там. Мы думали, что она ушла,...
                

In [14]:
#cleaning train
train['features']=train['features'].astype(str).apply(lower_case)
train['features']=train['features'].astype(str).apply(rem_tags)
#cleaning test
final_test['features']=final_test['features'].astype(str).apply(lower_case)
final_test['features']=final_test['features'].astype(str).apply(rem_tags)
print(train['features'])
print(final_test['features'])

<ipython-input-12-844a7fc59cc3>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(text, "html.parser").get_text()


0        and these comments were considered in formulat...
1        these are issues that we wrestle with in pract...
2        des petites choses comme celles-là font une di...
3        you know they can't really defend themselves l...
4        ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...
                               ...                        
12115    the results of even the most well designed epi...
12116    but there are two kinds of  the pleasure of do...
12117    the important thing is to realize that it's wa...
12118    at the west end is a detailed model of the who...
12119    for himself he chose atat??rk, or father of th...
Name: features, Length: 12120, dtype: object
0       بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...
1       هذا هو ما تم نصحنا به. عندما يتم إخبارهم بما ي...
2       et cela est en grande partie dû au fait que le...
3       与城市及其他公民及社区组织代表就ima的艺术发展进行对话& ima与其他组织合作，因为它们都...
4       она все еще была там. мы думали, что она ушла,...
                

In [15]:
# Training data preparation
from sklearn.model_selection import train_test_split
# Splitting the data
X_train, X_test, label_train, label_test = train_test_split(train['features'], train['label'], test_size=0.2, random_state=42)
#quick visualization
print(X_train)
print(X_test)

5289            اب اتنا خفیہ تھا یہ. یہ عوامی معلومات تھی۔
6647          oh  that's accommodating that is disruptive.
1245     more than anything else in this day and age th...
4417     aie! les boucaniers en-dessous s'écriaient, et...
5662     一个采购战略组织，是壮大的力资本发展战略的一部分, 这是在原则vi上讨论的。 原则四涉及财富...
                               ...                        
11964    analytical perspectives. perspectives can be a...
5191     the ile saint-louis is an enchanted self-conta...
5390           c'était mon désordre. j'ai fait une erreur.
860      the fcc will publish a notice in the federal r...
7270     kicked out of the house when she was only 16 (...
Name: features, Length: 9696, dtype: object
4960    кто? она спросила его с неожиданным интересом....
2088    others are zao (in tohoku) and a number of res...
396     trying to keep grass alive during a summer on ...
6385    so i guess my experience is is just with what ...
2619    the journal put the point succinctly to  is an...
                 

#MUSE (Multilingual Universal Sentence Encoder ) based from https://d-salvaggio.medium.com/multilingual-universal-sentence-encoder-muse-f8c9cd44f171

In [16]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-hub


In [17]:
!pip install tensorflow_text
import tensorflow_text
#encoding
import tensorflow as tf
import tensorflow_hub as hub

# Load the Multilingual Universal Sentence Encoder model
muse_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
muse_embed = hub.load(muse_url)

# Encode your text data (X) into embeddings
X_train_vec= muse_embed(X_train.tolist())  # X is a list of text data
X_test_vec= muse_embed(X_test.tolist())
final_test_vec=muse_embed(final_test['features'].tolist())

#Training

In [18]:
#Some models
from sklearn.svm import LinearSVC as SVM
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import classification_report as CR
from sklearn.naive_bayes import BernoulliNB as NB
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import Perceptron as PT
#initial models configuration
svm=SVM(penalty='l2',max_iter=500,C=1,random_state=42)
svc=SVC(max_iter=500,C=1,random_state=42)
lr=LR(max_iter=500)
nb=NB(alpha=1)
rfc=RFC(n_estimators=10,random_state=42)
pt=PT(tol=1e-3, random_state=42)
#array of models
models=[svm,lr, nb, rfc, pt]
#array of results
pred=[]

In [19]:
#Training with each model
from sklearn.metrics import accuracy_score, f1_score
def fit(x_train,y_train,x_test,y_test):
  for model in models:
    model.fit(x_train,y_train)
    print(model)
    prediction=(model.predict(x_test))
    print(CR(y_test,prediction))
    print("accuracy:",accuracy_score(y_test,prediction))
    pred.append(prediction)
#function execution
fit(X_train_vec,label_train,X_test_vec,label_test)

LinearSVC(C=1, max_iter=500, random_state=42)
              precision    recall  f1-score   support

           0       0.44      0.44      0.44       851
           1       0.37      0.34      0.35       773
           2       0.47      0.51      0.49       800

    accuracy                           0.43      2424
   macro avg       0.43      0.43      0.43      2424
weighted avg       0.43      0.43      0.43      2424

accuracy: 0.42945544554455445
LogisticRegression(max_iter=500)
              precision    recall  f1-score   support

           0       0.43      0.43      0.43       851
           1       0.38      0.35      0.36       773
           2       0.47      0.50      0.48       800

    accuracy                           0.43      2424
   macro avg       0.42      0.43      0.42      2424
weighted avg       0.42      0.43      0.43      2424

accuracy: 0.4265676567656766
BernoulliNB(alpha=1)
              precision    recall  f1-score   support

           0       0.42 

In [25]:
#Reconfiguring LinearSVC as it showed to be the best option
svm=SVM(penalty='l2',max_iter=2000,C=0.2,random_state=38,tol=1e-9)
svm.fit(X_train_vec,label_train)
y_pred=(svm.predict(X_test_vec))
print(CR(label_test,y_pred))
print("accuracy:",accuracy_score(label_test,y_pred))

              precision    recall  f1-score   support

           0       0.43      0.43      0.43       851
           1       0.38      0.34      0.36       773
           2       0.47      0.52      0.49       800

    accuracy                           0.43      2424
   macro avg       0.43      0.43      0.43      2424
weighted avg       0.43      0.43      0.43      2424

accuracy: 0.42945544554455445


#Execution

In [21]:
#prediction of the actual test data with linearSVC
y_pred=(svm.predict(final_test_vec))
#transforming into dataframe to facilitate csv creation

result_df=final_test.drop(columns=['lang_abv','language', 'premise', 'hypothesis', 'features'] )
result_df['prediction']=y_pred
print(result_df.head())

           id  prediction
0  c6d58c3f69           0
1  cefcc82292           2
2  e98005252c           1
3  58518c10ba           0
4  c32b0d16df           0


#CSV creation

In [22]:
from google.colab import files
#csv creation and download
file_name_csv = "submission.csv"  # Nombre del archivo CSV de salida
result_df.to_csv(file_name_csv, index=False)